In [1]:
#imports
import ccxt
import pandas as pd
import os
import time
from Historic_Crypto import HistoricalData
#from pytrends.request import TrendReq
#from twilio.rest import Client
from dotenv import load_dotenv
from MCForecastTools import MCSimulation
import panel as pn
from panel.interact import interact
import plotly.express as px
pn.extension("plotly")

In [2]:
# twilio_client = Client(twilio_account_sid, twilio_auth_token)
exchange_id = 'coinbasepro'


In [3]:
def retrieve_historical_data(product,start_date):
    return HistoricalData(product, 86400, start_date).retrieve_data()


In [4]:
eth = retrieve_historical_data("ETH-USD", "2019-10-01-00-00")
btc = retrieve_historical_data("BTC-USD", "2019-10-01-00-00")
# doge = retrieve_historical_data("DOGE-USD", "2019-10-01-00-00")
# ltc = retrieve_historical_data("LTC-USD", "2019-10-01-00-00")

Checking input parameters are in the correct format...
Formatting Dates...
Checking if ticker supplied is available on the CoinBase Pro API...
Connected to the CoinBase Pro API...
Ticker 'ETH-USD' found at the CoinBase Pro API, continuing to extraction...
Data for chunk 0 of 2 extracted
Data for chunk 1 of 2 extracted
Data for chunk 2 of 2 extracted
Checking input parameters are in the correct format...
Formatting Dates...
Checking if ticker supplied is available on the CoinBase Pro API...
Connected to the CoinBase Pro API...
Ticker 'BTC-USD' found at the CoinBase Pro API, continuing to extraction...
Data for chunk 0 of 2 extracted
Data for chunk 1 of 2 extracted
Data for chunk 2 of 2 extracted


In [5]:
eth

,low,high,open,close,volume
time,,,,,
2019-10-02,174.15,181.66,176.14,180.82,73478.776081
2019-10-03,169.59,181.26,180.90,175.05,70968.268332
2019-10-04,170.77,179.69,175.11,175.97,52275.848207
2019-10-05,172.27,177.05,175.97,176.73,32009.686893
2019-10-06,167.50,177.12,176.72,170.18,57631.704103
...,...,...,...,...,...
2021-07-07,2294.16,2411.19,2322.20,2316.82,159702.044084
2021-07-08,2084.00,2325.46,2317.64,2116.95,223830.003713
2021-07-09,2050.00,2189.28,2116.30,2147.38,190883.326179


In [6]:
btc

,low,high,open,close,volume
time,,,,,
2019-10-02,8168.85,8391.00,8321.87,8381.72,7016.971952
2019-10-03,8059.22,8419.95,8381.72,8240.41,7778.118423
2019-10-04,8002.21,8241.35,8241.27,8156.67,7715.280102
2019-10-05,8020.20,8200.00,8156.68,8147.63,3826.828707
2019-10-06,7780.85,8176.18,8147.63,7859.79,8697.939760
...,...,...,...,...,...
2021-07-07,33770.01,35077.46,34225.72,33878.56,9404.897987
2021-07-08,32111.00,33934.62,33879.50,32875.95,14147.352525
2021-07-09,32255.24,34100.00,32875.95,33824.26,8320.906493


In [7]:
eth.columns = pd.MultiIndex.from_product([['ETH'], eth.columns])
eth

ETH                                          
                low     high     open    close         volume
time                                                         
2019-10-02   174.15   181.66   176.14   180.82   73478.776081
2019-10-03   169.59   181.26   180.90   175.05   70968.268332
2019-10-04   170.77   179.69   175.11   175.97   52275.848207
2019-10-05   172.27   177.05   175.97   176.73   32009.686893
2019-10-06   167.50   177.12   176.72   170.18   57631.704103
...             ...      ...      ...      ...            ...
2021-07-07  2294.16  2411.19  2322.20  2316.82  159702.044084
2021-07-08  2084.00  2325.46  2317.64  2116.95  223830.003713
2021-07-09  2050.00  2189.28  2116.30  2147.38  190883.326179
2021-07-10  2075.00  2192.98  2147.38  2110.55  116594.485893
2021-07-11  2108.40  2137.22  2110.56  2135.24    2132.336018

[649 rows x 5 columns]

In [8]:
btc.columns = pd.MultiIndex.from_product([['BTC'], btc.columns])
btc

BTC                                            
                 low      high      open     close        volume
time                                                            
2019-10-02   8168.85   8391.00   8321.87   8381.72   7016.971952
2019-10-03   8059.22   8419.95   8381.72   8240.41   7778.118423
2019-10-04   8002.21   8241.35   8241.27   8156.67   7715.280102
2019-10-05   8020.20   8200.00   8156.68   8147.63   3826.828707
2019-10-06   7780.85   8176.18   8147.63   7859.79   8697.939760
...              ...       ...       ...       ...           ...
2021-07-07  33770.01  35077.46  34225.72  33878.56   9404.897987
2021-07-08  32111.00  33934.62  33879.50  32875.95  14147.352525
2021-07-09  32255.24  34100.00  32875.95  33824.26   8320.906493
2021-07-10  33027.83  34267.14  33824.25  33515.35   4906.245822
2021-07-11  33459.75  33805.15  33515.35  33752.94    141.604432

[649 rows x 5 columns]

In [9]:
#Creating a dataframe
portfolio_data = pd.concat([eth, btc], axis="columns", join="inner")
#portfolio_data.columns = [Eth,Btc]
portfolio_data


ETH                                                 BTC  \
                low     high     open    close         volume       low   
time                                                                      
2019-10-02   174.15   181.66   176.14   180.82   73478.776081   8168.85   
2019-10-03   169.59   181.26   180.90   175.05   70968.268332   8059.22   
2019-10-04   170.77   179.69   175.11   175.97   52275.848207   8002.21   
2019-10-05   172.27   177.05   175.97   176.73   32009.686893   8020.20   
2019-10-06   167.50   177.12   176.72   170.18   57631.704103   7780.85   
...             ...      ...      ...      ...            ...       ...   
2021-07-07  2294.16  2411.19  2322.20  2316.82  159702.044084  33770.01   
2021-07-08  2084.00  2325.46  2317.64  2116.95  223830.003713  32111.00   
2021-07-09  2050.00  2189.28  2116.30  2147.38  190883.326179  32255.24   
2021-07-10  2075.00  2192.98  2147.38  2110.55  116594.485893  33027.83   
2021-07-11  2108.40  2137.22  2110.56  2135.24    2132.336018  33459.75   

                                                        
                high      open     close        volume  
time                                                    
2019-10-02   8391.00   8321.87   8381.72   7016.971952  
2019-10-03   8419.95   8381.72   8240.41   7778.118423  
2019-10-04   8241.35   8241.27   8156.67   7715.280102  
2019-10-05   8200.00   8156.68   8147.63   3826.828707  
2019-10-06   8176.18   8147.63   7859.79   8697.939760  
...              ...       ...       ...           ...  
2021-07-07  35077.46  34225.72  33878.56   9404.897987  
2021-07-08  33934.62  33879.50  32875.95  14147.352525  
2021-07-09  34100.00  32875.95  33824.26   8320.906493  
2021-07-10  34267.14  33824.25  33515.35   4906.245822  
2021-07-11  33805.15  33515.35  33752.94    141.604432  

[649 rows x 10 columns]

In [10]:


# Configure a Monte Carlo simulation to forecast two year daily returns
MC_crypto = MCSimulation(
    portfolio_data = portfolio_data,
    weights = [.5,.5],
    num_simulation = 500,
    num_trading_days = 252*2
)

#Print the simulation input data 
MC_crypto.portfolio_data.head()

ETH                                                     \
               low    high    open   close        volume daily_return   
time                                                                    
2019-10-02  174.15  181.66  176.14  180.82  73478.776081          NaN   
2019-10-03  169.59  181.26  180.90  175.05  70968.268332    -0.031910   
2019-10-04  170.77  179.69  175.11  175.97  52275.848207     0.005256   
2019-10-05  172.27  177.05  175.97  176.73  32009.686893     0.004319   
2019-10-06  167.50  177.12  176.72  170.18  57631.704103    -0.037062   

                BTC                                                       
                low     high     open    close       volume daily_return  
time                                                                      
2019-10-02  8168.85  8391.00  8321.87  8381.72  7016.971952          NaN  
2019-10-03  8059.22  8419.95  8381.72  8240.41  7778.118423    -0.016859  
2019-10-04  8002.21  8241.35  8241.27  8156.67  7715.280102    -0.010162  
2019-10-05  8020.20  8200.00  8156.68  8147.63  3826.828707    -0.001108  
2019-10-06  7780.85  8176.18  8147.63  7859.79  8697.939760    -0.035328

In [11]:
# Run a Monte Carlo simulation to forecast five years cumulative returns
MC_df=MC_crypto.calc_cumulative_return()

In [12]:
MC_df

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,1.048637,0.940773,1.008011,0.969081,1.041555,1.036848,1.028312,0.985117,0.976897,1.029438,...,0.974026,1.040628,1.004459,0.991661,0.968242,0.999964,0.966033,0.974713,1.015614,0.983331
2,1.044361,0.908828,0.988024,1.002489,1.122952,1.058865,1.056370,0.995693,0.978715,1.056725,...,1.035694,1.049454,0.980853,0.990027,0.940729,0.992487,1.016361,0.996270,1.031683,0.975242
3,1.026472,0.863064,1.055476,1.047842,1.183372,1.101357,1.117919,0.981469,0.993091,1.059208,...,1.031584,1.044407,1.031200,0.997486,0.962986,0.952005,1.088531,0.988401,1.023995,0.985365
4,1.034575,0.869414,1.052108,1.057678,1.159026,1.041389,1.116557,1.002637,1.030547,1.030142,...,0.937553,1.049440,1.024425,0.972777,0.936543,0.997446,1.115929,0.990673,0.963417,0.998690
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,3.904512,23.834698,2.718993,8.060415,15.930024,7.224258,8.436029,6.027191,14.684115,10.803333,...,25.511413,5.837974,1.632694,6.999625,4.718221,2.121941,7.014574,4.933491,1.683565,17.065857
501,3.980347,22.846385,2.627999,7.898913,15.898070,7.481984,8.709828,6.171720,15.115010,10.908888,...,26.613673,5.706733,1.652948,7.011355,4.953629,2.063477,7.115408,5.123759,1.663967,16.212774
502,4.132568,22.661684,2.513595,7.691166,16.208560,7.570667,8.600153,6.381235,15.914069,11.273379,...,25.870410,5.578203,1.736550,7.261501,5.329406,2.158658,7.338376,5.274182,1.701785,16.227700
503,4.388311,23.930231,2.527424,8.119566,15.964833,7.726831,8.845686,6.360210,15.003795,10.730345,...,25.960848,5.700987,1.754047,7.425966,5.387849,2.198089,7.472635,5.379806,1.795042,16.757560


### Visualize the Results

In [ ]:
#Plot simulations
crypto_line_plot = MC_crypto.plot_simulation()
#df = px.data.gapminder().query("country=='Canada'")
#MC_df=MC_crypto
#fig=px.line(MC_crypto, x="days", y="Return", title="Cumulative Portfolio Return")
#fig.show()
#px.line(MC_df, x=MC_df.index, y=MC_df.columns)


In [ ]:
# Plot probability distribution and confidence intervals
dist_plot = MC_crypto.plot_distribution()

In [13]:
# Fetch summary statistics from the Monte Carlo simulation results
sum_stats = MC_crypto.summarize_cumulative_return()

# Print summary statistics
print(sum_stats)

count           500.000000
mean              8.724108
std               8.034295
min               0.384278
25%               3.756173
50%               6.281149
75%              10.788239
max              72.645073
95% CI Lower      1.347795
95% CI Upper     32.170374
Name: 504, dtype: float64


### Determine Upper and Lower Confidence Intervals

In [15]:
# Using lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our XXXX $15,000 investments in cryptocurrency
# even_ci_lower = round(sum_stats[8]*15000,2)
# even_ci_upper = round(sum_stats[9]*15000,2)

# Print results
# print(f"There is a 95% chance that an initial investment of $15,000 in the portfolio"
#       f" over the next 5 years will end within in the range of"
#       f" ${even_ci_lower} and ${even_ci_upper}.")
print(sum_stats[4])
print(sum_stats[5])
print(sum_stats[6])
print(sum_stats[8])
print(sum_stats[9])

3.756172573496902
6.281149212270545
10.788238657634745
1.347795323942959
32.17037382260294


In [22]:
def summarise_cumulative_returns(stats, seed_investment):
#     stats = mc_crypto.summarize_cumulative_return()
    stats_dict = {}
    stats_dict['25%'] = round(stats[4]*seed_investment,2)
    stats_dict['50%'] = round(stats[5]*seed_investment,2)
    stats_dict['75%'] = round(stats[7]*seed_investment,2)
    stats_dict['95%_lower'] = round(stats[8]*seed_investment,2)
    stats_dict['95%_upper'] = round(stats[9]*seed_investment,2)
    return pd.DataFrame.from_dict(stats_dict, orient='index')

In [23]:
summarise_cumulative_returns(sum_stats, 1000)

,0
25%,3756.17
50%,6281.15
75%,72645.07
95%_lower,1347.80
95%_upper,32170.37


In [25]:
pn.widgets.DataFrame(summarise_cumulative_returns(sum_stats, 1000), autosize_mode='fit_viewport')

DataFrame(autosize_mode='fit_viewport', value=              ...)